# Project: Wrangling and Analyze Data

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [2]:
import csv
import urllib.request
import pandas as pd
import tweepy
import os
import os.path
import requests
import json

In [72]:
def file_exists(base_p, file_name):
    return(os.path.exists(os.path.join(base_p, file_name)))

base_path = "C:/Users/mikle/Downloads/wgu/DA/project3"
#tw_arch_file = "twitter-archive-enhanced.csv"
#path_to_tw_arch_file = os.path.join(base_path, tw_arch_file)
#file_exists = os.path.exists(path_to_tw_arch_file)
if (not(file_exists(base_path,"twitter-archive-enhanced.csv"))):
    url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv'
    file_to_read = urllib.request.urlopen(url)
    df_ta = pd.read_csv(file_to_read)
    df_ta.to_csv(tw_arch_file)
else:
    file_to_read = path_to_tw_arch_file
    df_ta = pd.read_csv(file_to_read)
    print('debug; file red')
df_ta.head()
#df_ta.shape (2356, 18)


debug; file red


,Unnamed: 0,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [4]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = urllib.request.urlopen(url)
df_ipr = pd.read_csv(response, sep = '\t')
df_ipr.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


get all tweet_ids

In [33]:
tweet_ids = df_ta['tweet_id'].astype(str)

tweet_ids.index

RangeIndex(start=0, stop=2356, step=1)

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [9]:
consumer_key_filename = "consumer_key.txt"
consumer_secret_filename = "consumer_secret.txt"
access_token_filename = "access_token.txt"
access_secret_filename = "access_secret.txt"
bearer_token_filename = "bearer_token.txt"

path_to_consumer_key = os.path.join(base_path, consumer_key_filename)
path_to_consumer_secret = os.path.join(base_path, consumer_secret_filename)
path_to_access_token = os.path.join(base_path, access_token_filename)
path_to_access_secret = os.path.join(base_path, access_secret_filename)
path_to_bearer_token = os.path.join(base_path, bearer_token_filename)

fd1 = open(path_to_consumer_key , 'r')
fd2 = open(path_to_consumer_secret , 'r')
fd3 = open(path_to_access_token , 'r')
fd4 = open(path_to_access_secret , 'r')
fd5 = open(path_to_bearer_token , 'r')

consumer_key = fd1.read()
consumer_secret = fd2.read()
access_token = fd3.read()
access_secret = fd4.read()
bearer_token = fd5.read()

print (consumer_key)
print (consumer_secret)
print (access_token)
print (access_secret)
print (bearer_token)

fd1.close
fd2.close
fd3.close
fd5.close
fd4.close;

zgrKfmPIJIDvsayYvGnJ2zxtl
i5mjstQOAFuGF007T55uY8u3ojMIe5JSbW1veX0tGS3T81h0N7
971949013770514434-WhX45Rb33if4NLgpCsCLWQIh12v9TiJ
arps4VPLFkymxtVco5mTZbH9Zf4mVBDGQoZ2wbmglDkI9
AAAAAAAAAAAAAAAAAAAAAJnDXgEAAAAAK1aYkaupE8XVcun5jyYoQJE06BI%3DMiVObbjjQoTaiKpXeDXdodvnrXIiUacheSNrMKIln5PL5UpSfj


In [75]:
def get_tweat_url(tw_id):
    twitter_url = "https://api.twitter.com/2/tweets/" + str(tw_id)
    query_params = {#'id': tw_id,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source,text',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (twitter_url, query_params)
def get_tweats_url(tw_ids):
    twitter_url = "https://api.twitter.com/2/tweets"
    query_params = {'ids': tw_ids,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source,text',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (twitter_url, query_params)

def connect_to_twitter(url, headers, params, next_token = None):
    params['next_token'] = next_token
    resp = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(resp.status_code))
    if resp.status_code != 200:
        raise Exception(resp.status_code, resp.text)
    return resp.json()
  
    
def get_ids_string(start, quantity, tw_ids_series):
    result = ''
    #flag = False
    for i in range(start, start + quantity):
        if i in tw_ids_series.index:
            if len(result) > 0:
                result = result +','+ tw_ids_series[i]
            else:
                result = tw_ids_series[i]
        else:
            #flag = True
            break
    return (result, i-start+1)

current_cursor = 0
if (file_exists(base_path, 'tweet_json.txt')):
    with open('tweet_json.txt', 'r') as in_f: #if file exist, what is the length?
        current_cursor = len([0 for _ in in_f])
current_cursor 

6

In [76]:
with open('tweet_json.txt', 'a') as out_f:
    
    for k in range(0,3): #iterations quantity, later replace on while current cursor < max.shape
        ids_t = get_ids_string(current_cursor, 2, tweet_ids)
        url = get_tweats_url(ids_t[0])
        #print(url)
        headers = {"Authorization": "Bearer " + bearer_token}
        #print(headers)
        json_resp = connect_to_twitter(url[0], headers, url[1])
        for j in range(0, ids_t[1]):
            json.dump(json_resp.get('data')[j],out_f)
            out_f.write('\n')
        current_cursor += ids_t[1]
        #print(json.dumps(json_resp.get('data')[0], indent=2, sort_keys=True))

Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200


In [54]:
json_resp.get('data')[1]

{'source': 'Twitter for iPhone',
 'text': "This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",
 'conversation_id': '892177421306343426',
 'lang': 'en',
 'id': '892177421306343426',
 'reply_settings': 'everyone',
 'created_at': '2017-08-01T00:17:27.000Z',
 'author_id': '4196983835',
 'public_metrics': {'retweet_count': 5363,
  'reply_count': 172,
  'like_count': 29702,
  'quote_count': 274}}

ToDo: try to get tweets by batches of 10-100? pcs using the "/2/tweets" endpoint. Done
ToDo: implement limits on requests:
before every request to check existance file rqsts.csv
    if exist:
        load data to df : times all requests, check limits, 
                 if there over limit:
                     , stop with print
                 else:
                     make a request.
                     add time to df
                     If the oldest data is out of 15 minutes:
                         , delete it
                     re-write rqsts.csv
    else:
        make a request.
        add time to df
        create rqsts.csv
ToDo: implement saving af all jsons


In [31]:
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_secret)
#api = tweepy.API(auth)
client = tweepy.Client(consumer_key= consumer_key,consumer_secret= consumer_secret,access_token= access_token,access_token_secret= access_secret)
response = client.get_tweet(666029285002620928, user_auth=True)
print(type(response))
print(response.includes)
print((response.data.text))
print((response.meta))
#https://developer.twitter.com/en/docs/twitter-api/tweets/lookup/api-reference/get-tweets-id

<class 'tweepy.client.Response'>
{}
This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI
{}


## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



### Quality issues
1.

2.

3.

4.

5.

6.

7.

8.

### Tidiness issues
1.

2.

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
# Make copies of original pieces of data


### Issue #1:

#### Define:

#### Code

#### Test

### Issue #2:

#### Define

#### Code

#### Test

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization